In [ ]:
import numpy as np
import pandas as pd
from docplex.mp.model import Model
import matplotlib.pyplot as plt

In [ ]:
rnd=np.random
rnd.seed(0)

In [ ]:
data = pd.read_excel(r'C:\Users\Documents\project 3\experiment\LRFD.xlsx')
df = pd.DataFrame(data)

In [ ]:
nop=3
nol=3
nou=5
nott=6
R=[r for r in range (1,nop+1)]
J=[j for j in range (1,nol+1)]
I=[i for i in range (1,nou+1)]
T=[t for t in range (1,nott+1)]
nc=5000
mc=10000
ml=10000
bigM=10000000000

In [ ]:
RIT=[(r,i,t) for r in R for i in I for t in T]
#d={(r,i,t): 1 for r,i,t in RIT}
u={(r): 1000 for r in R}
k={(r):50 for r in R}
h={(r):20 for r in R}
pi={(r):300 for r in R}
l={(r):1 for r in R}
beta={(r):1 for r in R}
e={(j):10 for j in J}
v={(j):100 for j in J}
JK = [(j,k) for j in J for k in J if j != k]
lat = {(j, k): 80 for j, k in JK}
opr= {(j, k): 160 for j, k in JK}
IJ=[(i, j) for i in I for j in J ]
JKT=[(j,k,t) for j in J for k in J for t in T if j!=k ]
RJ=[(r,j) for r in R for j in J]
RT=[(r,t) for r in R for t in T]
RJT=[(r,j,t) for r in R for j in J for t in T]
RJKT=[(r,j,k,t) for r in R for j in J for k in J for t in T if j!=k]
JT=[(j,t) for j in J for t in T]

In [ ]:
excel= pd.ExcelFile(r'C:\Users\Documents\project 3\experiment\LRFD.xlsx')

In [ ]:
drit = pd.read_excel(excel,'drit')
demand_1= {k: row['drit'] for k, row in drit.iterrows()}
d={RIT[k]:demand_1[k] for k in demand_1}
fij = pd.read_excel(excel,'fij')
fij_1={k:row['fij'] for k, row in fij.iterrows()}
f={IJ[k]:fij_1[k] for k in fij_1}
print(f)

In [ ]:
mdl = Model('LRFOPT')

In [ ]:
X=mdl.binary_var_dict(IJ, name='X')
Y= mdl.continuous_var_dict(JKT, name='Y')
QC= mdl.continuous_var_dict(R, name='QC')
QL= mdl.continuous_var_dict(RJ, name='QL')
IC= mdl.continuous_var_dict(RT, name='IC')
BL= mdl.continuous_var_dict(RJT, name='BL')
BC= mdl.continuous_var_dict(RT, name='BC')
IL= mdl.continuous_var_dict(RJT, name='IL')
ZCL= mdl.continuous_var_dict(RJT, name='ZCL')
ZLAT= mdl.continuous_var_dict(RJKT, name='ZLAT')
RR= mdl.continuous_var_dict(RJT, name='RR')
QREP= mdl.continuous_var_dict(RT, name='QREP')
DD= mdl.continuous_var_dict(RJT, name='DD')
U= mdl.continuous_var_dict(JT, name='U')

In [ ]:
print(R)

In [ ]:
mdl.minimize(mdl.sum(f[i, j]*X[i, j] for i, j in IJ) 
             +mdl.sum(k[r]*QREP[r,t] for r,t in RT) 
             +mdl.sum(h[r]*IC[r,t] for r,t in RT) 
             +mdl.sum(h[r]*IL[r,j,t] for r,j,t in RJT) 
             +mdl.sum(u[r]*QL[r,j] for r,j in RJ) 
             +mdl.sum(u[r]*QC[r] for r in R) 
             +mdl.sum(v[j]*ZCL[r,j,t] for r,j,t in RJT) 
             +mdl.sum(lat[j,k]*ZLAT[r,j,k,t] for r,j,k,t in RJKT) 
             +mdl.sum(pi[r]*(BC[r,t]+BL[r,j,t]) for r,j,t in RJT) 
             +mdl.sum(opr[j,k]*Y[j,k,t] for j,k,t in JKT))

#constraint2
mdl.add_constraints(mdl.sum(u[r]*QC[r] for r in R) <= mc for r in R) 
#constraint3
mdl.add_constraints(mdl.sum(u[r]*QL[(r,j)] for r in R) <=ml for j in J) 
#constraint4
mdl.add_constraints(mdl.sum(X[(i,j)] for j in J) ==1 for i in I) 
#constraint5
mdl.add_constraints(mdl.sum(X[(i,j)] for i in I) >=1 for j in J)
#constraint6
mdl.add_constraints(mdl.sum(d[(r,i,t)]*3*X[(i,j)] for i in I) ==DD[(r,j,t)] for r,j,t in RJT) 
#constraint7
mdl.add_constraints(RR[(r,j,t)]+BL[(r,j,t)]==DD[(r,j,t)] for r,j,t in RJT if t==1)
mdl.add_constraints(RR[(r,j,t)]+BL[(r,j,t)]-BL[(r,j,t-1)]==DD[(r,j,t)] for r,j,t in RJT if t!=1)
#constraint8,9
mdl.add_constraints(QL[(r,j)]+ZCL[(r,j,t)]==RR[(r,j,t)]+IL[(r,j,t)] for r,j,t in RJT if t==1)
mdl.add_constraints(mdl.sum(ZLAT[(r,k,j,t)] for k in J if k!=j)+ZCL[(r,j,t)]+IL[(r,j,t-1)]==mdl.sum(ZLAT[(r,j,k,t)] for k in J if k!=j)+RR[(r,j,t)]+IL[(r,j,t)] for r,j,t in RJT if t!=1)
#constraint10
mdl.add_constraints(IL[(r,j,t)]<=QL[(r,j)] for r,j,t in RJT)
#constraint11
mdl.add_constraints(QC[(r)]==mdl.sum(ZCL[(r,j,t)] for j in J)+IC[(r,t)] for r,t in RT if t==1)
#constraint12
mdl.add_constraints(QREP[(r,(t-1))]+IC[(r,t-1)]+BC[(r,t)]-BC[(r,t-1)]==mdl.sum(ZCL[(r,j,t)] for j in J)+IC[(r,t)] for r,t in RT if t!=1)
#constraint13
mdl.add_constraints(IC[(r,t)]<=QC[(r)] for r,t in RT)

solution = mdl.solve(log_output=True)

In [ ]:
print(solution)

In [ ]:
mdl.get_solve_details()